In [2]:
import dash  # use Dash version 1.16.0 or higher for this app to work
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import plotly.express as px

df = px.data.gapminder()

C:\Users\DATASC~1\AppData\Local\Temp/ipykernel_1604/516561290.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\DATASC~1\AppData\Local\Temp/ipykernel_1604/516561290.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [3]:
df.head()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4


In [4]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Dropdown(id='dpdn2', value=['Germany','Brazil'], multi=True,
                 options=[{'label': x, 'value': x} for x in
                          df.country.unique()]),
    html.Div([
        dcc.Graph(id='pie-graph', figure={}, className='six columns'),
        dcc.Graph(id='my-graph', figure={}, clickData=None, hoverData=None, # I assigned None for tutorial purposes. By defualt, these are None, unless you specify otherwise.
                  config={
                      'staticPlot': False,     # True, False
                      'scrollZoom': True,      # True, False
                      'doubleClick': 'reset',  # 'reset', 'autosize' or 'reset+autosize', False
                      'showTips': False,       # True, False
                      'displayModeBar': True,  # True, False, 'hover'
                      'watermark': True,
                      # 'modeBarButtonsToRemove': ['pan2d','select2d'],
                        },
                  className='six columns'
                  )
    ])
])


@app.callback(
    Output(component_id='my-graph', component_property='figure'),
    Input(component_id='dpdn2', component_property='value'),
)
def update_graph(country_chosen):
    dff = df[df.country.isin(country_chosen)]
    fig = px.line(data_frame=dff, x='year', y='gdpPercap', color='country',
                  custom_data=['country', 'continent', 'lifeExp', 'pop'])
    fig.update_traces(mode='lines+markers')
    return fig


# Dash version 1.16.0 or higher
@app.callback(
    Output(component_id='pie-graph', component_property='figure'),
    Input(component_id='my-graph', component_property='hoverData'),
    Input(component_id='my-graph', component_property='clickData'),
    Input(component_id='my-graph', component_property='selectedData'),
    Input(component_id='dpdn2', component_property='value')
)
def update_side_graph(hov_data, clk_data, slct_data, country_chosen):
    if hov_data is None:
        dff2 = df[df.country.isin(country_chosen)]
        dff2 = dff2[dff2.year == 1952]
        print(dff2)
        fig2 = px.pie(data_frame=dff2, values='pop', names='country',
                      title='Population for 1952')
        return fig2
    else:
        print(f'hover data: {hov_data}')
        # print(hov_data['points'][0]['customdata'][0])
        # print(f'click data: {clk_data}')
        # print(f'selected data: {slct_data}')
        dff2 = df[df.country.isin(country_chosen)]
        hov_year = hov_data['points'][0]['x']
        dff2 = dff2[dff2.year == hov_year]
        fig2 = px.pie(data_frame=dff2, values='pop', names='country', title=f'Population for: {hov_year}')
        return fig2



if __name__ == '__main__':
    app.run_server(port=8888, use_reloader=False)

Dash is running on http://127.0.0.1:8888/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8888/ (Press CTRL+C to quit)


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\poc\lib\site-packages\dash\dash.py", line 1306, in dispatch
    cb = self.callback_map[output]
KeyError: 'the_graph.figure'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\poc\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\envs\poc\lib\site-packages\flask\app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\envs\poc\lib\site-packages\flask\app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\envs\poc\lib\site-packages\flask\app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\ProgramData\Anacond

127.0.0.1 - - [31/Mar/2022 15:33:36] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [31/Mar/2022 15:33:36] "GET /_favicon.ico?v=2.3.0 HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2022 15:33:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2022 15:33:40] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2022 15:33:40] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2022 15:33:40] "GET /_favicon.ico?v=2.3.0 HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2022 15:33:40] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2022 15:33:40] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2022 15:33:40] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


     country continent  year  lifeExp       pop    gdpPercap iso_alpha  \
168   Brazil  Americas  1952   50.917  56602560  2108.944355       BRA   
564  Germany    Europe  1952   67.500  69145952  7144.114393       DEU   

     iso_num  
168       76  
564      276  


127.0.0.1 - - [31/Mar/2022 15:33:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2022 15:33:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2022 15:33:44] "POST /_dash-update-component HTTP/1.1" 200 -


hover data: {'points': [{'curveNumber': 1, 'pointNumber': 4, 'pointIndex': 4, 'x': 1972, 'y': 18016.18027, 'bbox': {'x0': 1342.75, 'x1': 1348.75, 'y0': 248.08, 'y1': 254.08}, 'customdata': ['Germany', 'Europe', 71, 78717088]}]}
